In [3]:
import pandas as pd
import ast

In [4]:
RAW_DATA_PATH: str = 'data/dataset.raw.csv'
EXPORTED_DATA_PATH: str = 'data/dataset'

SAMPLE_SIZE: int = 1000
TRAIN_SPLIT: float = 0.8
RANDOM_STATE: int = 42

In [5]:
raw: pd.DataFrame = (
    pd.read_csv(
        RAW_DATA_PATH,
        skiprows=[1],
        index_col=[0],
        dtype={
            'text': 'string',
            'label': 'category'
        },
        converters={
            'span': ast.literal_eval
        }
    )
    .rename_axis('idx')
    .sample(SAMPLE_SIZE)
)
raw

,text,label,span
idx,,,
a692168b-ae0c-4641-9997-6cc7e0471aba,Masken im Flüchtlingslager,corona,"[0, 5]"
f7d4a734-d40a-4264-98ff-8ff7216c7097,Star Wars Masken Basteln Vorlagen Star Wars Ma...,web1,"[10, 15]"
b4ec27ff-d36f-44e2-8619-d0e383d054b3,"Ganz frisch aus der Recherche, möchte ich hier...",web1,"[91, 96]"
2801fbec-6f32-40ed-8128-a7ca642d425f,Mareo öffnete die Augen und sah eine Gestalt d...,web2,"[63, 68]"
85617ff0-a3a2-4318-a780-a061935944d1,"„85 Prozent der Menschen, die eine Maske trage...",corona,"[35, 40]"
...,...,...,...
0edcbda2-20dc-419b-b6c2-9011563fbfab,"Sie holt mich von der Arbeit ab, wir fahren zu...",corona,"[79, 84]"
f9b88985-e57a-4cce-866c-fab8fe6c6b4e,"( Empfehlung: Hals Decollte, bei Stauungen Lym...",web2,"[82, 87]"
19a0eb4f-f831-494c-b306-8e7cdf9a5d6f,"Die Polizeisprecherin sagte, wenn die Abstands...",corona,"[91, 96]"


In [6]:
raw.dtypes

text     string[python]
label          category
span             object
dtype: object

In [7]:
raw.label.unique()

['corona', 'web1', 'web2']
Categories (3, object): ['corona', 'web1', 'web2']

In [8]:
raw.label.value_counts()

label
web2      345
corona    329
web1      326
Name: count, dtype: int64

In [9]:
(
    raw
    .apply(lambda row: row.text[row.span[0]:row.span[1]], axis=1)
    .value_counts()
)

Maske    1000
Name: count, dtype: int64

In [10]:
train_split = raw.sample(
    frac=TRAIN_SPLIT, 
    random_state=RANDOM_STATE
)
train_split.to_parquet(f'{EXPORTED_DATA_PATH}/train.parquet')

In [11]:
test_split = raw.loc[
    raw.index.difference(train_split.index)
]
test_split.to_parquet(f'{EXPORTED_DATA_PATH}/test.parquet')